In [1]:
import pytimber
ldb = pytimber.LoggingDB()

Specify the output file and the times you want to download. Timber and pyTimber conflict with regards to the the times, probably because of winter and summertime. If you want to have data stored in Timber from 00:00 to 01:00, you might to request either from 01:00 to 02:00 or even 02:00 to 03:00. We have to account for this shift later on.

In [2]:
# %load ../ionsrcopt/source_features.py
class SourceFeatures(object):
    TIMESTAMP = 'UTC_TIME'
    BIASDISCAQNV = 'IP.NSRCGEN:BIASDISCAQNV'
    GASAQN = 'IP.NSRCGEN:GASAQN'
    GASSASAQN = 'IP.NSRCGEN:GASSASAQN'
    SOLINJ_CURRENT = 'IP.SOLINJ.ACQUISITION:CURRENT'
    SOLCEN_CURRENT = 'IP.SOLCEN.ACQUISITION:CURRENT'
    SOLEXT_CURRENT = 'IP.SOLEXT.ACQUISITION:CURRENT'
    OVEN1AQNP = 'IP.NSRCGEN:OVEN1AQNP'
    OVEN2AQNP = 'IP.NSRCGEN:OVEN2AQNP'
    SOURCEHTAQNI = 'IP.NSRCGEN:SOURCEHTAQNI'
    SAIREM2_FORWARDPOWER = 'IP.SAIREM2:FORWARDPOWER'
    THOMSON_FORWARDPOWER = 'IP.NSRCGEN:RFTHOMSONAQNFWD'
    BCT05_CURRENT = 'ITL.BCT05:CURRENT'
    BCT25_CURRENT = 'ITF.BCT25:CURRENT'

Now select all parameters you are interested in.

In [13]:
def get_result(parameters_raw, parameters_scaled, t1, t2):
    print("Loading Data in interval {} to {}".format(t1, t2))
    result = {}

    if parameters_raw:
        result = ldb.get(parameters_raw, t1, t2, unixtime=True)

    for k, v in parameters_scaled.items():
        data = ldb.getScaled(k, t1, t2, scaleAlgorithm=v['scale'], scaleInterval=v['interval'], scaleSize=v['size'], unixtime=True)
        result.update(data)
    
    return result

In [14]:
import pandas as pd
from os import path

def load_existing_data(filename, replace_column):
    if not path.exists(filename):
        print("The file {} does not yet exist, we will create a new one".format(filename))
        return pd.DataFrame(columns=[SourceFeatures.TIMESTAMP])
    
    print("Loading data from {}.".format(filename))
    if replace_column:
        print("We will replace columns that already exist")
    else:
        print("We will only append new columns")
        
    df = pd.read_csv(filename)
    return df

def create_base_df(filename, replace_file, replace_column):
    if replace_file:
        df = pd.DataFrame(columns=[SourceFeatures.TIMESTAMP])
    else:
        df = load_existing_data(filename, replace_column)

    df.set_index(SourceFeatures.TIMESTAMP, inplace = True)
    df.index = pd.to_datetime(df.index).tz_localize('UTC')
    
    return df

In [15]:
def check_duplicate_times(time_series):
    x = time_series.duplicated()
    count = x[x].count()
    if count > 0:
        print("Time duplicates exist!")

def join_result(df, result):
    print("Joining together result")
    for parameter, values in result.items():
        print("For column {} {} datapoints exist.".format(parameter, len(values[1])))

        if parameter in df.columns:
            print("Parameter {} is already in the data frame. There it has {} values. In the newly retrieved dataset it has {} values.".format(parameter, df[parameter].count(), len(values[1])))
            if not replace_column:
                print("Skipping.")
                continue
            else:
                print("Removing old column.")
                df = df.drop(parameter, axis=1)
                df = df.dropna(axis=0, how='all')

        df_column = pd.DataFrame(columns=[SourceFeatures.TIMESTAMP, parameter])
        df_column[SourceFeatures.TIMESTAMP] = pd.Series([datetime.fromtimestamp(timestamp, tz=pytz.utc) for timestamp in values[0]])
        check_duplicate_times(df_column[SourceFeatures.TIMESTAMP])
        df_column[parameter] = values[1]

        df_column.set_index(SourceFeatures.TIMESTAMP, inplace = True)
        df_column.dropna(inplace=True)

        df = df.join(df_column, how='outer')

    df = df.reindex(sorted(df.columns), axis=1)
    df.index = df.index.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
    df.index.name = SourceFeatures.TIMESTAMP
    return df

If the timeindex is duplicated, we will only keep the first occurence.

And save the output to the file.

In [22]:
def save_df(df, filename):
    print("Saving result to {}".format(filename))
    df.to_csv(filename)

In [17]:
def get_data(filename, t1, t2, parameters_raw, parameters_scaled, replace_file, replace_column):
    result = get_result(parameters_raw, parameters_scaled, t1, t2)
    df = create_base_df(filename, replace_file, replace_column)
    df = join_result(df, result)
    df = df[~df.index.duplicated(keep='first')].copy()
    save_df(df, filename)

In [18]:
parameters_raw = [
        SourceFeatures.BIASDISCAQNV, 
        SourceFeatures.GASAQN, 
        SourceFeatures.OVEN1AQNP,
        SourceFeatures.SOLINJ_CURRENT,
        SourceFeatures.SOLCEN_CURRENT,
        SourceFeatures.SOLEXT_CURRENT,
        SourceFeatures.SOURCEHTAQNI,
        SourceFeatures.BCT25_CURRENT
]
parameters_scaled = {
        SourceFeatures.THOMSON_FORWARDPOWER : {'scale' : 'AVG', 'interval' : 'SECOND', 'size' : '10'},
        SourceFeatures.BCT05_CURRENT : {'scale' : 'AVG', 'interval' : 'MINUTE', 'size' : '2'}
}

In [21]:
import pandas as pd
from datetime import datetime
import pytz

def load_data(filename, year, month):
    t1 = '{}-{:02d}-01 00:00:00.000'.format(year, month)
    if month == 12:
        month = 0
        year += 1
    
    t2 = '{}-{:02d}-01 00:00:00.000'.format(year, month+1)

    t1 = pytz.utc.localize(datetime.strptime(t1, '%Y-%m-%d %H:%M:%S.%f')).astimezone(tz=None)
    t2 = pytz.utc.localize(datetime.strptime(t2, '%Y-%m-%d %H:%M:%S.%f')).astimezone(tz=None)
    replace_file = False
    replace_column = True
    get_data(filename, t1, t2, parameters_raw, parameters_scaled, replace_file, replace_column)
    print("Finished download of data {}/{}\n".format(month, year))

In [23]:
output_folder = '../Data_Raw/'

year = 2015
start_month = 'Apr'
end_month = 'Nov'

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for m in months[months.index(start_month):months.index(end_month)+1]:
    filename = output_folder + '{}{}.csv'.format(m, year)
    load_data(filename, year, months.index(m)+1)

Loading Data in interval 2015-04-01 02:00:00+02:00 to 2015-05-01 02:00:00+02:00


The file ../Data_Raw/Apr2015.csv does not yet exist, we will create a new one
Joining together result
For column IP.NSRCGEN:BIASDISCAQNV 8440 datapoints exist.
For column IP.NSRCGEN:GASAQN 9186 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 122 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 9383 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 733 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 64718 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 128396 datapoints exist.
For column ITF.BCT25:CURRENT 205405 datapoints exist.
For column IP.NSRCGEN:RFTHOMSONAQNFWD 259200 datapoints exist.
For column ITL.BCT05:CURRENT 21600 datapoints exist.
Saving result to ../Data_Raw/Apr2015.csv
Finished download of data 4/2015

Loading Data in interval 2015-05-01 02:00:00+02:00 to 2015-06-01 02:00:00+02:00


The file ../Data_Raw/May2015.csv does not yet exist, we will create a new one
Joining together result
For column IP.NSRCGEN:BIASDISCAQNV 11128 datapoints exist.
For column IP.NSRCGEN:GASAQN 10159 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 11554 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 177733 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 1104 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 167049 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 316453 datapoints exist.
For column ITF.BCT25:CURRENT 11177 datapoints exist.
For column IP.NSRCGEN:RFTHOMSONAQNFWD 267840 datapoints exist.
For column ITL.BCT05:CURRENT 22320 datapoints exist.
Saving result to ../Data_Raw/May2015.csv
Finished download of data 5/2015

Loading Data in interval 2015-06-01 02:00:00+02:00 to 2015-07-01 02:00:00+02:00


The file ../Data_Raw/Jun2015.csv does not yet exist, we will create a new one
Joining together result
For column IP.NSRCGEN:BIASDISCAQNV 8975 datapoints exist.
For column IP.NSRCGEN:GASAQN 10538 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 8805 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 269186 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 1962 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 68253 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 153504 datapoints exist.
For column ITF.BCT25:CURRENT 146328 datapoints exist.
For column IP.NSRCGEN:RFTHOMSONAQNFWD 259200 datapoints exist.
For column ITL.BCT05:CURRENT 21600 datapoints exist.
Saving result to ../Data_Raw/Jun2015.csv
Finished download of data 6/2015

Loading Data in interval 2015-07-01 02:00:00+02:00 to 2015-08-01 02:00:00+02:00


The file ../Data_Raw/Jul2015.csv does not yet exist, we will create a new one
Joining together result
For column IP.NSRCGEN:BIASDISCAQNV 9144 datapoints exist.
For column IP.NSRCGEN:GASAQN 9560 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 8968 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 277255 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 1783 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 16552 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 26292 datapoints exist.
For column ITF.BCT25:CURRENT 402543 datapoints exist.
For column IP.NSRCGEN:RFTHOMSONAQNFWD 267840 datapoints exist.
For column ITL.BCT05:CURRENT 22320 datapoints exist.
Saving result to ../Data_Raw/Jul2015.csv
Finished download of data 7/2015

Loading Data in interval 2015-08-01 02:00:00+02:00 to 2015-09-01 02:00:00+02:00


The file ../Data_Raw/Aug2015.csv does not yet exist, we will create a new one
Joining together result
For column IP.NSRCGEN:BIASDISCAQNV 9202 datapoints exist.
For column IP.NSRCGEN:GASAQN 10208 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 8901 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 288107 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 3968 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 14566 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 23223 datapoints exist.
For column ITF.BCT25:CURRENT 276972 datapoints exist.
For column IP.NSRCGEN:RFTHOMSONAQNFWD 267840 datapoints exist.
For column ITL.BCT05:CURRENT 22320 datapoints exist.
Saving result to ../Data_Raw/Aug2015.csv
Finished download of data 8/2015

Loading Data in interval 2015-09-01 02:00:00+02:00 to 2015-10-01 02:00:00+02:00


The file ../Data_Raw/Sep2015.csv does not yet exist, we will create a new one
Joining together result
For column IP.NSRCGEN:BIASDISCAQNV 8951 datapoints exist.
For column IP.NSRCGEN:GASAQN 18400 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 8590 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 250216 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 2007 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 17698 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 24504 datapoints exist.
For column ITF.BCT25:CURRENT 294851 datapoints exist.
For column IP.NSRCGEN:RFTHOMSONAQNFWD 259200 datapoints exist.
For column ITL.BCT05:CURRENT 21600 datapoints exist.
Saving result to ../Data_Raw/Sep2015.csv
Finished download of data 9/2015

Loading Data in interval 2015-10-01 02:00:00+02:00 to 2015-11-01 01:00:00+01:00


The file ../Data_Raw/Oct2015.csv does not yet exist, we will create a new one
Joining together result
For column IP.NSRCGEN:BIASDISCAQNV 9140 datapoints exist.
For column IP.NSRCGEN:GASAQN 11981 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 8913 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 250183 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 1902 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 17097 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 25956 datapoints exist.
For column ITF.BCT25:CURRENT 231242 datapoints exist.
For column IP.NSRCGEN:RFTHOMSONAQNFWD 267840 datapoints exist.
For column ITL.BCT05:CURRENT 22320 datapoints exist.
Saving result to ../Data_Raw/Oct2015.csv
Finished download of data 10/2015

Loading Data in interval 2015-11-01 01:00:00+01:00 to 2015-12-01 01:00:00+01:00


The file ../Data_Raw/Nov2015.csv does not yet exist, we will create a new one
Joining together result
For column IP.NSRCGEN:BIASDISCAQNV 8461 datapoints exist.
For column IP.NSRCGEN:GASAQN 8993 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 8555 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 249023 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 1567 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 16907 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 25889 datapoints exist.
For column ITF.BCT25:CURRENT 261350 datapoints exist.
For column IP.NSRCGEN:RFTHOMSONAQNFWD 259200 datapoints exist.
For column ITL.BCT05:CURRENT 21600 datapoints exist.
Saving result to ../Data_Raw/Nov2015.csv
Finished download of data 11/2015

